In [55]:
from pymongo import MongoClient
conn = "mongodb://127.0.0.1:27017"
client = MongoClient(conn)
db_connection = client["titanic_db"]
collection_name = "passangers"

In [56]:
import pandas as pd
passanger = db_connection[collection_name]

In [57]:
df = pd.read_csv('titanic.csv')

In [58]:
df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [59]:
import json
from bson import json_util
from bson.objectid import ObjectId
from typing import Dict, List
from datetime import timedelta


data_dict = df.to_dict(orient='records')
collection = db_connection.get_collection(collection_name)
collection.insert_many(data_dict)


InsertManyResult([ObjectId('68392168dd66e7ea66c60054'), ObjectId('68392168dd66e7ea66c60055'), ObjectId('68392168dd66e7ea66c60056'), ObjectId('68392168dd66e7ea66c60057'), ObjectId('68392168dd66e7ea66c60058'), ObjectId('68392168dd66e7ea66c60059'), ObjectId('68392168dd66e7ea66c6005a'), ObjectId('68392168dd66e7ea66c6005b'), ObjectId('68392168dd66e7ea66c6005c'), ObjectId('68392168dd66e7ea66c6005d'), ObjectId('68392168dd66e7ea66c6005e'), ObjectId('68392168dd66e7ea66c6005f'), ObjectId('68392168dd66e7ea66c60060'), ObjectId('68392168dd66e7ea66c60061'), ObjectId('68392168dd66e7ea66c60062'), ObjectId('68392168dd66e7ea66c60063'), ObjectId('68392168dd66e7ea66c60064'), ObjectId('68392168dd66e7ea66c60065'), ObjectId('68392168dd66e7ea66c60066'), ObjectId('68392168dd66e7ea66c60067'), ObjectId('68392168dd66e7ea66c60068'), ObjectId('68392168dd66e7ea66c60069'), ObjectId('68392168dd66e7ea66c6006a'), ObjectId('68392168dd66e7ea66c6006b'), ObjectId('68392168dd66e7ea66c6006c'), ObjectId('68392168dd66e7ea66c600

In [60]:
def insert_document(document):
        collection = db_connection.get_collection(collection_name)
        collection.insert_one(document)
        return document

In [61]:
def select_many_gt(field, value):
        collection = db_connection.get_collection(collection_name)
        data = collection.find({
            field : {"$gt" : value}}
        )

        response = []
        for elem in data: response.append(elem)

        return response


In [62]:
def select_one(field, value):
        collection = db_connection.get_collection(collection_name)
        response = collection.find_one({field:value})
        return response

In [63]:
def select_if_property_exists(field):
        collection = db_connection.get_collection(collection_name)
        data = collection.find({field : { "$exists": True } })
        for elem in data: print(elem)

In [64]:
def delete_registry(field, value):
    collection = db_connection.get_collection(collection_name)
    data = collection.delete_one({field: value})
    print(data.deleted_count)


In [65]:
select_one("Ticket", "373450")

{'_id': ObjectId('68392159dd66e7ea66c5fcdc'),
 'PassengerId': 5,
 'Survived': 0,
 'Pclass': 3,
 'Name': 'Allen, Mr. William Henry',
 'Sex': 'male',
 'Age': 35.0,
 'SibSp': 0,
 'Parch': 0,
 'Ticket': '373450',
 'Fare': 8.05,
 'Cabin': nan,
 'Embarked': 'S'}

In [66]:
select_if_property_exists("Cabin")

{'_id': ObjectId('68392159dd66e7ea66c5fcd8'), 'PassengerId': 1, 'Survived': 0, 'Pclass': 3, 'Name': 'Braund, Mr. Owen Harris', 'Sex': 'male', 'Age': 22.0, 'SibSp': 1, 'Parch': 0, 'Ticket': 'A/5 21171', 'Fare': 7.25, 'Cabin': nan, 'Embarked': 'S'}
{'_id': ObjectId('68392159dd66e7ea66c5fcd9'), 'PassengerId': 2, 'Survived': 1, 'Pclass': 1, 'Name': 'Cumings, Mrs. John Bradley (Florence Briggs Thayer)', 'Sex': 'female', 'Age': 38.0, 'SibSp': 1, 'Parch': 0, 'Ticket': 'PC 17599', 'Fare': 71.2833, 'Cabin': 'C85', 'Embarked': 'C'}
{'_id': ObjectId('68392159dd66e7ea66c5fcda'), 'PassengerId': 3, 'Survived': 1, 'Pclass': 3, 'Name': 'Heikkinen, Miss. Laina', 'Sex': 'female', 'Age': 26.0, 'SibSp': 0, 'Parch': 0, 'Ticket': 'STON/O2. 3101282', 'Fare': 7.925, 'Cabin': nan, 'Embarked': 'S'}
{'_id': ObjectId('68392159dd66e7ea66c5fcdb'), 'PassengerId': 4, 'Survived': 1, 'Pclass': 1, 'Name': 'Futrelle, Mrs. Jacques Heath (Lily May Peel)', 'Sex': 'female', 'Age': 35.0, 'SibSp': 1, 'Parch': 0, 'Ticket': '113

In [67]:
resultado = pd.DataFrame(select_many_gt("Fare", 15))
print(resultado)

                          _id  PassengerId  Survived  Pclass  \
0    68392159dd66e7ea66c5fcd9            2         1       1   
1    68392159dd66e7ea66c5fcdb            4         1       1   
2    68392159dd66e7ea66c5fcde            7         0       1   
3    68392159dd66e7ea66c5fcdf            8         0       3   
4    68392159dd66e7ea66c5fce1           10         1       2   
..                        ...          ...       ...     ...   
861  68392168dd66e7ea66c603c4          881         1       2   
862  68392168dd66e7ea66c603c9          886         0       3   
863  68392168dd66e7ea66c603cb          888         1       1   
864  68392168dd66e7ea66c603cc          889         0       3   
865  68392168dd66e7ea66c603cd          890         1       1   

                                                  Name     Sex   Age  SibSp  \
0    Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
1         Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0     

In [68]:
insert_document({
    'PassengerId': 1,
    'Survived': 0,
    'Pclass': 3,
    'Name': 'Braund, Mr. Owen Harris',
    'Sex': 'male',
    'Age': 22,
    'SibSp': 1,
    'Parch': 0,
    'Ticket': 'A/5 21171',
    'Fare': 7.25,
    'Cabin': '',
    'Embarked': 'S'
})


{'PassengerId': 1,
 'Survived': 0,
 'Pclass': 3,
 'Name': 'Braund, Mr. Owen Harris',
 'Sex': 'male',
 'Age': 22,
 'SibSp': 1,
 'Parch': 0,
 'Ticket': 'A/5 21171',
 'Fare': 7.25,
 'Cabin': '',
 'Embarked': 'S',
 '_id': ObjectId('68392168dd66e7ea66c603cf')}

In [69]:
delete_registry("PassengerId", 14)


1


In [70]:
df = pd.DataFrame(list(passanger.find()))
df

,_id,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,68392159dd66e7ea66c5fcd8,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,68392159dd66e7ea66c5fcd9,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,68392159dd66e7ea66c5fcda,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,68392159dd66e7ea66c5fcdb,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,68392159dd66e7ea66c5fcdc,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1777,68392168dd66e7ea66c603cb,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
1778,68392168dd66e7ea66c603cc,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
1779,68392168dd66e7ea66c603cd,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C
1780,68392168dd66e7ea66c603ce,891,0,3,"Dooley, Mr. Patrick",male,32.0,0,0,370376,7.7500,NaN,Q
